# 응용통계학 (6주차) 4월 14일
> 모형 진단

- toc:true
- branch: master
- badges: true
- comments: false
- author: 최서연
- categories: [Applied Statistics, 모형 진단]

## 응용통계학 : 모형 진단

선형모형에서 추정량의 특정 성질이나 이후 추론과정들은 모형의 가정에 의존하게 된다. 즉, 가정에 대한 검토과정이 정당성을 획득하기 위해 필요한 절차이다. 모형 가정에 대한 이슈는 다음과 같이 요약될 수 있다.
- $y = X\beta + \epsilon$
1. 오차항 가정
2. 모형가정

1. 오차항 : 정규성, 등분산성, 독립성
    - stochastic part; Stochastic refers to the property of being well described by a random probability distribution.
2. 모형 : $ E(y)=Xβ $ : 선형성
    - deterministric part = symmetric part
3. 특이 관측치 : 간혹 소수의 관측치들이 모형에 잘 맞지 않아 모형 전체에 영향을 미칠 수 있다.
    - outlier 있나?
    - 어떻게 처리해서 모델을 봐 볼까?

## Checking error assumptions

오차항을 관측 불가하므로 잔차에 기반하여 가정을 검토한다.

$$e=y−X\hat{β}=(I−H)y.$$

오차항은 서로 독립이더라도 잔차들은 그렇지 않음을 유의하자.

### constant variance (등분산성)

$e_i vs \hat{y}_i$(x축 vs y축) 산점도(residual plot)가 가장 기본이 되는 진단 방법이다. 만약 가정이 모두 만족된다면 수직축 기준으로 대칭적이고 일정한 정도를 보이는 변동만이 나타날 것이다.

- Note : 가로축에 설명변수를 넣은 산점도를 이용하기도 한다.

![](https://i.stack.imgur.com/rtn7e.png)

In [3]:
library(faraway)

In [4]:
data(savings,package="faraway")
str(savings)

'data.frame':	50 obs. of  5 variables:
 $ sr   : num  11.43 12.07 13.17 5.75 12.88 ...
 $ pop15: num  29.4 23.3 23.8 41.9 42.2 ...
 $ pop75: num  2.87 4.41 4.43 1.67 0.83 2.85 1.34 0.67 1.06 1.14 ...
 $ dpi  : num  2330 1508 2108 189 728 ...
 $ ddpi : num  2.87 3.93 3.82 0.22 4.56 2.43 2.67 6.51 3.08 2.8 ...
